In [1]:
import nltk
from tqdm import tqdm
import os
import html
import re

import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, fpmax

from fyp.crypto import Crypto


In [ ]:
# https://link.springer.com/content/pdf/10.1007/11892755_87.pdf


In [2]:
crypto = Crypto()
base = '/its/home/ep396/Documents/FYP/'
name = "dataset"


In [3]:
def load_db(name, base):
    e = base + f"encrypted_{name}.db"
    d = base + f"decrypted_{name}.db"
    crypto.age_decrypt_file(e, d)


In [4]:
def unload_db(name, base):
    e = base + f"encrypted_{name}.db"
    d = base + f"decrypted_{name}.db"
    crypto.age_encrypt_file(d, e)

    os.remove(d)


In [5]:
load_db(name, base)


In [6]:
from fyp.db_dataset import Tweet, ReferencedTweet, DataSplit


In [7]:
nltk.download("stopwords")
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package stopwords to
[nltk_data]     /its/home/ep396/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /its/home/ep396/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /its/home/ep396/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /its/home/ep396/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [8]:
def initial_phase(initial_set_s):
    tweet_data_set = []

    for tweet_text in initial_set_s:
        cleaned_text = clean_text(tweet_text)
        eal = pre_processing_words(cleaned_text)
        tweet_data_set.append(eal)
        
    return tweet_data_set


In [9]:
wnl = nltk.stem.WordNetLemmatizer()
sw = nltk.corpus.stopwords.words('english')


In [10]:
def pre_processing_words(text):
    tokenized = nltk.word_tokenize(text)
    return tokenized


In [11]:
def clean_text(text):
    html.unescape(text)
    removed_links = re.sub(r"https?://\S+", '', text)
    removed_mentions = re.sub(r"(^|[^@\w])@(\w{1,15})\b", '', removed_links)
    return removed_mentions


In [12]:
init_user_author_id = Tweet.select().first().author_id
init_user_tweets_query = Tweet.select(Tweet.text).where(Tweet.author_id == init_user_author_id)
init_user_corpus = [tweet.text for tweet in tqdm(init_user_tweets_query)]


100%|██████████| 1581/1581 [00:00<00:00, 334773.56it/s]


In [13]:
(sum([len(text) for text in init_user_corpus]) // len(init_user_corpus)) * 0.025


6.775

In [14]:
frequency_threshold = (sum([len(text) for text in init_user_corpus]) // len(init_user_corpus)) * 0.025


In [15]:
tweet_data_set = initial_phase(init_user_corpus)


In [16]:
# https://towardsdatascience.com/how-to-find-closed-and-maximal-frequent-itemsets-from-fp-growth-861a1ef13e21
# https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/fpgrowth/


In [17]:
te = TransactionEncoder()
te_ary = te.fit(tweet_data_set).transform(tweet_data_set)
df = pd.DataFrame(te_ary, columns=te.columns_)


In [18]:
frequent = fpgrowth(df, min_support=0.01, use_colnames=True)


In [31]:
su = frequent.support.unique()


In [20]:
frequent


,support,itemsets
0,0.841872,"(,)"
1,0.808349,(.)
2,0.611638,(to)
3,0.531942,(a)
4,0.344719,(it)
...,...,...
1051310,0.010120,"(,, a, spouse, to)"
1051311,0.010120,"(,, trans, to, spouse, a)"
1051312,0.010120,"(a, trans, the, spouse)"
1051313,0.010120,"(,, a, the, spouse)"


In [24]:
max_frequent = fpmax(df, min_support=0.05, use_colnames=True)


In [28]:
max_su = max_frequent.support.unique()


In [25]:
max_frequent


,support,itemsets
0,0.050601,(now)
1,0.051233,(could)
2,0.054396,(here)
3,0.050601,"(., They)"
4,0.052498,"(If, ,)"
...,...,...
4968,0.057559,"(and, ,, that, to, ., a, the)"
4969,0.068311,"(and, ,, trans, that, to, ., a)"
4970,0.076534,"(and, ,, trans, that, ., a, the)"
4971,0.060721,"(,, trans, that, to, ., of, a, the)"


In [33]:
freq_dict = {}
for i in tqdm(range(len(su))):
    inset2 = list(frequent.loc[frequent.support<=su[i]]['itemsets'])
    freq_dict[su[i]] = inset2


 96%|█████████▌| 534/559 [03:04<00:21,  1.19it/s]

: 

: 

In [ ]:
ml = []
for index, row in frequent.iterrows():
    isclose = True
    cli = row['itemsets']
    cls = row['support']
    checkset = freq_dict[cls]
    for i in checkset:
        if (cli!=i):
            if(frozenset.issubset(cli,i)):
                isclose = False
                break
    
    if(isclose):
        ml.append(row['itemsets'])
        

KeyError: 0.8418722327640734

In [ ]:
ml
